# SELFLEARNING Prompt Engineering Techniques

Notebook available at: https://github.com/Bluedata-Consulting/GAAI-B5-GCP/blob/main/SELFLEARNING/code4_Prompt_Engineering_Techniques.ipynb


Prompt engineering is the process of designing and refining prompts for large language models (LLMs) to elicit desired responses. Different techniques exist to improve the model's performance and controllability.

In [1]:
from google import genai
from google.genai.types import HttpOptions
client = genai.Client(http_options=HttpOptions(api_version='v1'))

In [19]:

from google.genai import types

# creating a function to get outcome
def generate_response(prompt,model='gemini-2.0-flash',temperature=0):
  response = client.models.generate_content(
    model=model,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=temperature,  # Set the temperature here
        max_output_tokens=1000,
    ))

  return response.text


In [3]:
generate_response("Hi! How are you.")

"Hi! I'm doing well, thank you for asking. How are you today?\n"


## Zero-Shot Prompting
**Description:** In zero-shot prompting, the model is given a prompt with a task and no examples of how to perform it. The model is expected to perform the task based solely on its pre-training.

**Application Areas:** Simple tasks where the model has a strong understanding from its training data, such as classification, translation, or summarization for common domains.

**When to Use:**
- When the task is straightforward and aligns well with the model's pre-training.
- When collecting examples for few-shot prompting is difficult or time-consuming.
- As a baseline to see how well the model performs without explicit examples.

**Example:**
Prompt: "Translate the following English sentence to French: 'Hello, how are you?'"



In [4]:
# zero shot prompting
prompt = f"""
Create a conversation story between child and grandparent.
"""
response = generate_response(prompt)
print(response)

"Grandpa, look!" Lily, all of six years old, burst through the back door, her pigtails bouncing. She held a dandelion, its fluffy head threatening to disintegrate with every excited movement.

Grandpa Joe, perched on the porch swing with a well-worn book, looked up, a smile crinkling the corners of his eyes. "Well, hello there, little dandelion fairy. What treasure have you brought me?"

Lily skipped closer, holding the dandelion out


In [5]:
prompt = """Classify the customer's request into one of the following categories:
- Account Balance Inquiry
- Loan Application Status
- Report Lost Card
- Fund Transfer Issue
- Credit Card Application


Customer: "Can you tell me what to do in case I misplaced my debit card?"
Category:"""
output = generate_response(prompt)
print(output)

Report Lost Card



## Few-Shot Prompting
**Description:** Few-shot prompting involves providing the model with a few examples (typically 1 to 5) of the desired input-output pairs before presenting the actual task. This helps the model understand the format and type of output expected.

**Application Areas:** Tasks where the desired output format is specific, or the task requires understanding a particular style or pattern. Examples include generating text in a specific style, answering questions in a particular format, or performing tasks on less common data.

**When to Use:**
- When zero-shot performance is not sufficient.
- When the task requires a specific output format or style.
- When the task involves domains or data less frequently seen during pre-training.
- To guide the model towards desired behavior without fine-tuning.
**Example:**
Prompt:
"Translate to Spanish:
English: 'Goodbye'
Spanish: 'Adiós'
English: 'Thank you'
Spanish: 'Gracias'
English: 'Please'
Spanish: 'Por favor'
English: 'Excuse me'
Spanish: 'Disculpe'"


In [6]:
# few shot prompting
prompt = f"""
your task is to answer in a consistent sytle:

<child>: Teach me about patience.
<grandma>: Patience is like waiting for whole day to see the moon in the evening and then sleeping after having a look at it.

<child>: Teach me about resilience.
"""
response = generate_response(prompt)
print(response)

<grandma>: Resilience is like a seed pushing through the hard ground to become a flower, even after a long winter.



In [8]:
prompt = """Classify the customer's request into one of the following categories:
- Account Balance Inquiry
- Loan Application Status
- Report Lost Card
- Fund Transfer Issue
- Credit Card Application

Examples:
Customer: "What's the current balance in my savings account?"
Category: Account Balance Inquiry

Customer: "I need to know where my loan application stands."
Category: Loan Application Status

Customer: "My credit card is missing, I think I lost it."
Category: Report Lost Card

Customer: "I tried to send money to my friend but it's not going through."
Category: Fund Transfer Issue

Customer: "I want to apply for a new credit card."
Category: Credit Card Application

Customer: "Can you tell me how much money I have in my checking account?"
Category:"""

output = generate_response(prompt)
print(output)

Account Balance Inquiry



In [9]:
def get_categorization(qus):
    prompt = f"""Classify the customer's request into one of the following categories:
      - Account Balance Inquiry
      - Loan Application Status
      - Report Lost Card
      - Fund Transfer Issue
      - Credit Card Application

      Examples:
      Customer: "What's the current balance in my savings account?"
      Category: Account Balance Inquiry

      Customer: "I need to know where my loan application stands."
      Category: Loan Application Status

      Customer: "My credit card is missing, I think I lost it."
      Category: Report Lost Card

      Customer: "I tried to send money to my friend but it's not going through."
      Category: Fund Transfer Issue

      Customer: "I want to apply for a new credit card."
      Category: Credit Card Application

      Customer: {qus}
      Category:"""
    output = generate_response(prompt)
    return output


In [10]:
get_categorization("can you help me with my checkin balance?")

'Account Balance Inquiry\n'

In [11]:
get_categorization("I think I missed my credit card while travelling yesterday at airport, what do i do next?")

'Report Lost Card\n'

In [12]:
get_categorization("How do apply for a new credit card?")

'Credit Card Application\n'


## Chain-of-Thought (CoT) Prompting
**Description:** CoT prompting involves explicitly asking the model to show its reasoning steps or thought process before arriving at the final answer. This is often achieved by including phrases like "Let's think step by step" in the prompt. CoT can be used in a zero-shot or few-shot setting (Zero-shot CoT or Few-shot CoT).

**Application Areas:** Complex reasoning tasks such as arithmetic, logical deduction, multi-step problem-solving, and question answering that requires synthesis of information.

**When to Use:**
- When the task requires multi-step reasoning.
- When the final answer depends on intermediate steps.
- To improve the accuracy on complex problems by guiding the model through the solution process.
- To understand the model's reasoning and debug incorrect outputs.
**Example (Few-shot CoT):**
Prompt:
"Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. He bought 2 cans with 3 balls each, so he bought 2 * 3 = 6 balls. Total balls = 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 and then bought 6 more, how many apples do they have?
A: Let's think step by step. The cafeteria had 23 apples. They used 20, so they had 23 - 20 = 3 apples left. Then they bought 6 more, so they have 3 + 6 = 9 apples. The answer is 9.

Q: [New complex question]
A: Let's think step by step."


In [13]:
# Without Chain of Thought Prompting = Only Few Shot (No CoT)
prompt = f"""
Answer below question
Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:
"""
response = generate_response(prompt)
print(response)

Student: 8 pens



In [14]:
# with Chain of Thought prompting = Few Shot CoT
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: Johan had 5 apples, 4 boxes with 5 apples each = 5 + 4*5 = 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Marry had 5 pens, gave 3 to her friend, bought 2 boxes of pen with 3 pen in each box = 5 - 3 + 2*3 = 8 pens



In [15]:
# with Chain of Thought prompting = Zero shot CoT
prompt = f"""

Think step by step to answer below question, do not come to conclusion without following process.
Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Okay, let's break this problem down step-by-step:

1. **Start:** Mary begins with 5 pens.

2. **Gives away:** She gives 3 pens away.  So we subtract: 5 - 3 = 2 pens.

3. **Boxes of pens:** She buys 2 boxes of pens.

4. **Pens per box:** Each box has 3 pens.

5. **Total new pens:** So she gets


In [20]:
# without Chain of thought

prompt = """
Based on the following eligibility criteria, determine which tier of the Premium Checking Account (Gold, Platinum, or None) the customer qualifies for.
Answer the question:
**Gold Tier Eligibility:**
- Maintain a minimum average daily balance of $5,000 OR
- Have at least $25,000 in combined linked accounts.
- Must also have at least 5 debit card transactions per month.

**Platinum Tier Eligibility:**
- Maintain a minimum average daily balance of $25,000 OR
- Have at least $100,000 in combined linked accounts.
- Must also have at least 10 debit card transactions per month.
- Additionally, must have a credit score of 700 or higher.

**Customer Profile:**
- Average daily checking balance: $8,000
- Combined linked accounts balance: $45,000
- Debit card transactions last month: 17
- Credit score: 720

Which tier does this customer qualify for?
"""
print(generate_response(prompt))

Let's analyze the customer's profile against each tier's requirements:

*   **Gold Tier:**
    *   Average daily balance: $8,000 (Meets the $5,000 requirement)
    *   Combined linked accounts: $45,000 (Meets the $25,000 requirement)
    *   Debit card transactions: 17 (Meets the 5 transaction requirement)
    *   **Conclusion: The customer meets all Gold Tier requirements.**

*   **Platinum Tier:**
    *   Average daily balance: $8,000 (Does NOT meet the $25,000 requirement)
    *   Combined linked accounts: $45,000 (Does NOT meet the $100,000 requirement)
    *   Debit card transactions: 17 (Meets the 10 transaction requirement)
    *   Credit score: 720 (Meets the 700 or higher requirement)
    *   **Conclusion: The customer does NOT meet all Platinum Tier requirements (specifically the balance requirements).**

**Answer:** Gold



In [21]:
# Zero shot Chain of thought

prompt = """
Based on the following eligibility criteria, determine which tier of the Premium Checking Account (Gold, Platinum, or None) the customer qualifies for.
Answer the question:
**Gold Tier Eligibility:**
- Maintain a minimum average daily balance of $5,000 OR
- Have at least $25,000 in combined linked accounts.
- Must also have at least 5 debit card transactions per month.

**Platinum Tier Eligibility:**
- Maintain a minimum average daily balance of $25,000 OR
- Have at least $100,000 in combined linked accounts.
- Must also have at least 10 debit card transactions per month.
- Additionally, must have a credit score of 700 or higher.

**Customer Profile:**
- Average daily checking balance: $8,000
- Combined linked accounts balance: $15,000
- Debit card transactions last month: 7
- Credit score: 720

Which tier does this customer qualify for?
Lets analyze the customer profile step by step and try answer the question.
"""
print(generate_response(prompt))

Okay, let's break down the customer's eligibility for each tier:

**Gold Tier:**

*   **Minimum Average Daily Balance:** The customer has $8,000, which meets the $5,000 requirement.
*   **Combined Linked Accounts:** The customer has $15,000, which does NOT meet the $25,000 requirement.
*   **Debit Card Transactions:** The customer had 7 transactions, which meets the 5 transaction requirement.

Since the customer meets the average daily balance requirement and the debit card transaction requirement, they qualify for the Gold Tier.

**Platinum Tier:**

*   **Minimum Average Daily Balance:** The customer has $8,000, which does NOT meet the $25,000 requirement.
*   **Combined Linked Accounts:** The customer has $15,000, which does NOT meet the $100,000 requirement.
*   **Debit Card Transactions:** The customer had 7 transactions, which does NOT meet the 10 transaction requirement.
*   **Credit Score:** The customer has a credit score of 720, which meets the 700 or higher requirement.

Sinc

## Tree of Thoughts (ToT)

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.



**Description:** ToT is an extension of CoT where the model explores multiple reasoning paths (a tree structure) instead of a single linear chain of thought. It generates multiple intermediate thoughts and explores their consequences, allowing for backtracking and exploration of different possibilities before selecting the most promising path to the solution.

**Application Areas:** More complex and open-ended reasoning tasks than CoT, such as creative writing, strategic planning, and problem-solving that might require exploring multiple hypotheses.

**When to Use:**
- When the problem has multiple potential approaches or intermediate steps.
- When a single linear chain of thought might lead to local optima or dead ends.
- For tasks requiring more extensive exploration of possibilities.

**How it Works (conceptually):** ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

Generate potential intermediate thoughts -> Evaluate their validity/promise -> Select the most promising ones -> Recursively generate further thoughts until a solution is reached. This typically requires more sophisticated prompting or external control mechanisms than basic CoT.


In [22]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

Okay, let's get our expert panel together to solve this watch mystery! We have:

*   **Expert A: The Probability Analyst:** Focuses on the likelihood of events based on Carlos's actions.
*   **Expert B: The Motion Physicist:** Considers the physics of the watch's movement and trajectory.
*   **Expert C: The Human Behaviorist:** Analyzes Carlos's actions and likely thought processes.
*   **Expert D: The Loss Prevention Specialist:** Thinks about common loss scenarios and patterns.
*   **Expert E: The Empathetic Reconstructor:** Tries to mentally reconstruct Carlos's journey and experience.

Let's begin!

**Step 1:**

*   **Expert A (Probability Analyst):** The watch was definitely in the towel at the locker room. The first key event is the shaking of the towel at the lounger. This is the first point where the watch *could* have been dislodged. Therefore, the lounger area has a high initial probability. (Likelihood: 70%)

*   **Expert B (Motion Physicist):** The vigorous shaking at the l

In [23]:

# ToT Prompt for Loan Application Approval

prompt = """
Imagine 5 different loan officers are reviewing a customer's loan application.
They will brainstorm the decision step by step, reasoning carefully and taking all facts into consideration from the application.
Each loan officer will write down 1 key observation or analysis step from the application data,
then share it with the group.
They will each critique their response, and all the responses of others based on standard lending practices and risk assessment criteria.
They will check their analysis against established bank policies and economic indicators.
They will keep going through steps until they reach their conclusion on whether to approve or reject the loan, taking into account the observations and critiques of the other loan officers.
If at any time they realize there is a flaw in their analysis or a crucial piece of information is overlooked, they will backtrack to where that flaw occurred.
If any loan officer realizes their assessment is weak or incorrect at any point, they acknowledge this and start another train of thought, focusing on different aspects of the application.
Each loan officer will assign a likelihood (e.g., High Risk, Medium Risk, Low Risk) based on their current assertion about the application's overall risk profile.
Continue until the loan officers agree on the single most likely outcome (Approve or Reject) based on a consensus risk assessment.

The customer's loan application details are as follows:

Customer Name: [Customer Name]
Loan Amount Requested: [Amount]
Loan Type: [e.g., Personal Loan, Mortgage]
Annual Income: [Income]
Credit Score: [Score]
Employment Status: [e.g., Full-time, Self-employed]
Employment Duration: [Duration]
Existing Debt Obligations: [List or summary]
Assets: [List or summary]
Purpose of Loan: [Purpose]
Payment History: [Summary of past credit behavior]

Based on these details, where is the single most likely outcome of the loan application (Approve or Reject)?
"""

# Replace bracketed information with actual application details
customer_details_prompt = prompt.replace("[Customer Name]", "John Doe") \
                               .replace("[Amount]", "$50,000") \
                               .replace("[Loan Type]", "Personal Loan") \
                               .replace("[Income]", "$75,000") \
                               .replace("[Score]", "720") \
                               .replace("[Employment Status]", "Full-time") \
                               .replace("[Employment Duration]", "5 years") \
                               .replace("[Existing Debt Obligations]", "Credit card balance: $5,000, Auto loan payment: $400/month") \
                               .replace("[Assets]", "Savings account: $20,000, Car value: $15,000") \
                               .replace("[Purpose of Loan]", "Home renovation") \
                               .replace("[Summary of past credit behavior]", "No late payments in the last 3 years")

ans = generate_response(customer_details_prompt)

print(ans)


Okay, let's simulate this loan application review process with five loan officers.

**Loan Application Summary:**

*   **Customer Name:** John Doe
*   **Loan Amount Requested:** $50,000
*   **Loan Type:** Personal Loan
*   **Annual Income:** $75,000
*   **Credit Score:** 720
*   **Employment Status:** Full-time
*   **Employment Duration:** 5 years
*   **Existing Debt Obligations:** $1,000/month (includes car loan and credit card payments)
*   **Assets:** $20,000 in savings, $10,000 in investments
*   **Purpose of Loan:** Home Improvement
*   **Payment History:** No late payments in the last 3 years

**Round 1: Initial Observations**

*   **Loan Officer 1 (LO1):** "The credit score of 720 is good, indicating a responsible borrower. This is a positive initial indicator." (Likelihood: Low Risk)
*   **Loan Officer 2 (LO2):** "Income of $75,000 against a $50,000 loan request needs further scrutiny. We need to calculate the Debt-to-Income (DTI) ratio to see how manageable this loan is." (Lik

In [24]:
# ToT Prompt for Enterprise Banking Scenario: Supply Chain Finance Optimization

prompt = """
Imagine 5 different experts in Supply Chain Finance, Risk Management, Data Science, Operations, and Customer Relationship Management are collaborating to address the following challenge:

A large manufacturing company, a client of your bank, is experiencing significant working capital strain due to long payment terms with their suppliers (averaging 90 days)
and relatively short payment terms from their buyers (averaging 45 days).
This creates a cash flow gap that they currently bridge using expensive short-term loans from other institutions.

Your bank wants to propose a comprehensive Supply Chain Finance (SCF) solution that benefits both the manufacturing company (the buyer) and their key suppliers,
 increases the bank's revenue through new products, and strengthens the overall client relationship.
 The goal is not just to offer a standard factoring or reverse factoring product, but to design an integrated, optimized solution leveraging data and
 potentially incorporating risk mitigation strategies for the bank.

The experts will brainstorm the design of this SCF solution step by step, reasoning carefully and taking all relevant factors into consideration for the buyer,
their suppliers, and the bank.

All experts will write down 1 step of their thinking related to designing this optimized SCF solution, then share it with the group.
They will each critique their response, and all the responses of others.
They will consider the perspectives and constraints of the buyer (manufacturing company), their diverse set of suppliers (ranging from large corporations to small businesses), and the bank (risk appetite, operational capacity, revenue goals).
They will keep going through steps until they reach a well-defined and compelling SCF solution proposal, taking into account the thoughts of the other experts.
If at any time they realize that there is a flaw in their logic or a critical aspect is missed, they will backtrack to where that flaw occurred.
If any expert realizes they're wrong at any point, they acknowledge this and start another train of thought.
Each expert will assign a likelihood of their current proposed element of the solution being successful or viable.

Continue until the experts agree on the single most effective and viable integrated SCF solution proposal that addresses the working capital needs of the buyer and their suppliers while creating a valuable new revenue stream and strengthening the relationship for the bank.

The challenge is:
Design an optimized and integrated Supply Chain Finance solution for a large manufacturing client facing a working capital gap due to mismatched payment terms, considering the needs of their suppliers and the bank's objectives.
"""

ans = generate_response(prompt)
print(ans)


Okay, let's begin. Here's how the five experts will approach this challenge:

**The Experts:**

*   **Supply Chain Finance Expert (SCF):** Focuses on the structure and mechanics of the SCF program.
*   **Risk Management Expert (RM):** Identifies and mitigates risks for the bank.
*   **Data Science Expert (DS):** Leverages data to optimize the program and assess risk.
*   **Operations Expert (OP):** Ensures the program is operationally feasible and scalable.
*   **Customer Relationship Management Expert (CRM):** Focuses on client and supplier onboarding, communication, and satisfaction.

**Round 1: Initial Thoughts**

Each expert will write down their initial thought/step.

*   **SCF:** "First, we need to understand the supplier base segmentation. Are we dealing with a few large suppliers and many small ones, or a more even distribution? This will heavily influence the type of SCF program we can offer. We should start with a dynamic discounting program for the larger suppliers and then 

## Chain-of-Verification (CoVe) Prompting

**Description:** Chain-of-Verification (CoVe) is a prompting technique designed to improve the factual accuracy of Large Language Models (LLMs) by explicitly prompting the model to generate a draft response, then generate verification questions for its own response, answer those questions independently of the initial draft, and finally use the answers to revise and improve the original draft. This creates a self-correction mechanism.

**Application Areas:** Tasks requiring high factual accuracy and reduced hallucination, such as question answering, summarization of factual information, and generating reports or summaries based on provided text. It is particularly useful when dealing with complex or potentially ambiguous information where the model might otherwise confidently generate incorrect details.

**When to Use:**
- When factual accuracy is critical and errors could be harmful or misleading.
- To reduce hallucinations and improve the reliability of the model's output.
- When verifying information against external sources is not immediately feasible or desired within the prompting process itself.
- For generating summaries or answers that require synthesizing information from multiple potential points within the model's knowledge or context.

**How it Works:**
1. **Generate Initial Draft:** The model first generates a direct answer or response to the original prompt.
2. **Generate Verification Questions:** The model is then prompted to generate questions that would help verify the correctness of the statements made in the initial draft.
3. **Answer Verification Questions:** The model answers these verification questions *independently* of the initial draft, drawing from its internal knowledge or provided context. This step is crucial as it prevents the model from simply confirming its own potential errors.
4. **Revise Initial Draft:** The model compares the answers to the verification questions with the original draft and revises the draft to align with the verified information.

This iterative process helps the model identify inconsistencies or inaccuracies in its initial output and correct them before presenting the final response.

**Example (Conceptual):**

Prompt: "Explain the process of photosynthesis and its main outputs."

1.  **Initial Draft (Internal):** "Photosynthesis is how plants make food. They use sunlight, water, and carbon dioxide. The main output is oxygen and sugar." (Might contain slight inaccuracies or missing details).
2.  **Generate Verification Questions (Internal):**
    *   "What are the inputs of photosynthesis?"
    *   "What are the outputs of photosynthesis?"
    *   "What is the role of chlorophyll?"
    *   "Where does photosynthesis primarily occur in a plant?"
3.  **Answer Verification Questions (Internal, independently):**
    *   "Inputs are carbon dioxide, water, and light energy."
    *   "Outputs are glucose (a sugar) and oxygen."
    *   "Chlorophyll is a pigment that absorbs light energy."
    *   "It occurs mainly in chloroplasts within plant cells."
4.  **Revise Initial Draft (Internal, based on verified answers):** "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll. During photosynthesis, plants take in carbon dioxide from the air, water from the soil, and light energy from the sun. Using the energy absorbed by chlorophyll in their chloroplasts, they convert these inputs into glucose, a sugar they use for energy and growth, and oxygen, which is released as a byproduct."

The final output presented to the user would be the revised draft. While the user doesn't see the intermediate steps, the process happens internally within the model's processing based on the CoVe prompt structure.


In [25]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response to check for accuracy.
Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer.
Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt)

print(ans)


Okay, let's do this.

**Initial Response:**

Here are some athletes born in the United States:

*   Michael Jordan (Basketball)
*   Serena Williams (Tennis)
*   LeBron James (Basketball)
*   Simone Biles (Gymnastics)
*   Tom Brady (American Football)

**Verification Questions and Answers:**

1.  **Question:** Was Michael Jordan born in the United States?
    **Answer:** Yes. He was born in Brooklyn, New York.

2.  **Question:** Was Serena Williams born in the United States?
    **Answer:** Yes. She was born in Saginaw, Michigan.

3.  **Question:** Was LeBron James born in the United States?
    **Answer:** Yes. He was born in Akron, Ohio.

4.  **Question:** Was Simone Biles born in the United States?
    **Answer:** Yes. She was born in Columbus, Ohio.

5.  **Question:** Was Tom Brady born in the United States?
    **Answer:** Yes. He was born in San Mateo, California.

**Revised and Verified Response:**

Here are some athletes born in the United States:

*   Michael Jordan (Basketball

In [26]:
prompt = """
In an enterprise banking context, a customer is requesting a complex derivative trade.
This involves intricate details regarding the underlying asset, notional value, tenor, strike price, option type (if applicable),
counterparty credit risk, regulatory compliance, and internal risk limits.

The task is to assess the viability and risks of this specific derivative trade request for the bank.

First, generate an initial assessment of the trade request, including potential benefits and immediate red flags based on standard derivative trading practices
and market knowledge.

Then, create and answer a series of verification questions specifically tailored to validate the critical details and associated risks mentioned in the initial assessment.
 Think through each question carefully, drawing upon knowledge of financial instruments, risk management frameworks, regulatory requirements (e.g., Dodd-Frank, EMIR),
 and internal banking policies.

Examples of potential verification questions:
- Is the counterparty's credit risk within acceptable limits for a trade of this size and tenor?
- Does this trade comply with all relevant regulatory requirements for derivative transactions?
- Does the pricing of this derivative accurately reflect current market conditions and volatility?
- How does this trade impact the bank's overall exposure to the underlying asset?
- Are the operational processes and systems in place to handle the settlement and ongoing management of this specific derivative type?
- Has the client undergone the necessary suitability and appropriateness checks for complex products like this?

After answering each verification question independently, consider these answers and revise the initial assessment to formulate a final,
 verified decision on whether to proceed with the trade, including any necessary conditions or risk mitigation steps.
 Ensure the final response is a comprehensive analysis reflecting the accuracy and findings from the verification process and aligns with the bank's risk appetite and policies.

The customer's derivative trade request details are:
[Provide specific details of the trade request here, e.g.,
Underlying Asset: EUR/USD currency pair
Notional Value: $100,000,000
Tenor: 2 years
Trade Type: FX Forward
Client: [Client Name], Credit Rating: [Rating]
Purpose: Hedging foreign exchange exposure]
"""

# Replace bracketed information with actual trade request details
trade_details_prompt = prompt.replace("[Provide specific details of the trade request here...]",
                                       """Underlying Asset: Interest Rate Swap (Fixed-to-Floating USD)
Notional Value: $250,000,000
Tenor: 5 years
Trade Type: Receive Fixed, Pay Floating
Client: [Client Name], Credit Rating: AA-
Purpose: Hedge against rising interest rates on existing floating-rate debt""")

ans = generate_response(trade_details_prompt)
print(ans)


Okay, let's break down this complex derivative trade request.

**Customer's Derivative Trade Request Details:**

*   **Underlying Asset:** EUR/USD currency pair
*   **Notional Value:** $100,000,000
*   **Tenor:** 2 years
*   **Trade Type:** FX Forward
*   **Client:** Acme Corp, Credit Rating: BBB+
*   **Purpose:** Hedging foreign exchange exposure

**I. Initial Assessment:**

**Potential Benefits:**

*   **Client Relationship:** Facilitating this trade strengthens the relationship with Acme Corp, a BBB+ rated client.
*   **Revenue Generation:** The bank earns a spread on the FX forward.
*   **Hedging Solution:** Provides Acme Corp with a solution to manage their FX risk, potentially leading to further business.

**Immediate Red Flags:**

*   **Large Notional Value:** $100 million is a significant amount, requiring careful consideration of credit risk and market impact.
*   **Two-Year Tenor:** A two-year tenor introduces longer-term exposure to currency fluctuations and counterparty ris

## ReAct Prompting
**Description:** ReAct (Reasoning and Acting) prompting is a technique that combines reasoning trace generation with task-specific actions. It encourages the model to interleave thinking steps (Reasoning) with taking actions (Acting) through tools or interacting with an environment. The model observes the outcomes of its actions and uses this information to refine its subsequent reasoning and actions. This allows the model to perform tasks that require interacting with external sources of information or performing physical (or simulated physical) actions.

**Application Areas:** Tasks that require external knowledge lookup, interacting with APIs or tools, complex decision-making, and environments where actions have observable consequences. Examples include answering questions that require searching the web, using a calculator for complex math, interacting with a simulated game environment, or completing tasks that involve multiple external steps.

**When to Use:**
- When the task requires up-to-date information not present in the model's training data.
- When the task involves complex calculations or operations that the model is not inherently good at performing reliably.
- When the task requires interacting with external systems or data sources.
- To enable the model to break down a complex task into smaller, executable steps based on external feedback.

**How it Works (Simulated in a Prompt):** In a prompt, ReAct is simulated by guiding the model to explicitly state its "Thought" process (reasoning about the next step or action) and its "Action" (the action it would take or the tool it would use), followed by an "Observation" (the simulated result of the action). The model then uses the Observation to inform its next Thought and Action.


In [27]:
prompt = """
You are a helpful assistant that can answer questions by first thinking step-by-step about the question, then deciding on an action (like searching or using a tool),
observing the result of the action, and then repeating the process until you can formulate a final answer.

Your process should follow this format:

Thought: Think step-by-step about the question and what information you need to answer it.
Action: [Describe the action you would take or tool you would use]
Observation: [Simulate the result of the action]
... (Repeat Thought, Action, Observation until you have enough information)
Thought: Now that I have the necessary information, I can formulate the final answer.
Final Answer: [Provide the final answer]

Here is the question: What is the current population of Tokyo?

"""

ans = generate_response(prompt)
print(ans)


Thought: To find the current population of Tokyo, I need to consult a reliable source of demographic data. I will start by searching the web for the most recent population estimates for Tokyo. I will look for sources like the Tokyo Metropolitan Government, the United Nations, or reputable statistical websites.
Action: Search the web for "current population of Tokyo"
Observation: The search results show several sources, including:
1. Worldometers: Claims a population of 14,043,084 (2023)
2. Macrotrends: Claims a population of 37,435,000 (2023)
3. Tokyo Metropolitan Government: (Link to official website)

I need to investigate the Tokyo Metropolitan Government website to find the most accurate and official number.
Action: Visit the Tokyo Metropolitan Government website and search for population statistics.
Observation: After navigating the Tokyo Metropolitan Government website (simulated), I found a page with population statistics. The latest data available is from January 1, 2024, which

In [28]:

prompt = """
You are an expert financial analyst assistant. You need to determine if a company's stock is overvalued based on its current P/E ratio compared to the industry average.

Follow the ReAct process:

Thought: Think step-by-step about how to assess if the stock is overvalued based on the P/E ratio.
Action: [Describe the action to find the company's current stock price and EPS to calculate its P/E ratio]
Observation: [Simulate the result, e.g., "Company X Stock Price: $150, EPS: $5.00"]
Thought: Calculate the company's P/E ratio.
Action: [Describe the calculation: Stock Price / EPS]
Observation: [Simulate the result, e.g., "Company X P/E Ratio: 30x"]
Thought: Find the industry average P/E ratio for comparison.
Action: [Describe the action to find the industry average P/E ratio for the relevant sector (e.g., Technology)]
Observation: [Simulate the result, e.g., "Industry Average P/E Ratio (Technology): 25x"]
Thought: Compare the company's P/E ratio to the industry average and determine if it is overvalued.
Action: [Describe the comparison and decision process]
Observation: [Simulate the result of the comparison]
Thought: Formulate the final assessment.
Final Answer: [Provide the final assessment based on the P/E comparison]

Here is the company: Company X (Technology Sector)
"""

ans = generate_response(prompt)
print(ans)

**Thought:** To determine if Company X's stock is overvalued based on its P/E ratio, I need to first calculate the company's P/E ratio and then compare it to the average P/E ratio of the technology sector. A significantly higher P/E ratio than the industry average suggests the stock might be overvalued.

**Action:** I will use a financial data provider (simulated here) to find Company X's current stock price and its Earnings Per Share (EPS).

**Observation:** Company X Stock Price: $150, EPS: $5.00

**Thought:** Now I need to calculate Company X's P/E ratio using the stock price and EPS.

**Action:** I will calculate the P/E ratio by dividing the stock price by the EPS: $150 / $5.00

**Observation:** Company X P/E Ratio: 30x

**Thought:** Next, I need to find the average P/E ratio for the technology sector to compare it with Company X's P/E ratio.

**Action:** I will use a financial data provider (simulated here) to find the industry average P/E ratio for the Technology sector.

**Obse

## Conversational App

In [ ]:
# Write code





You:  exit


## Thank You